<a href="https://colab.research.google.com/github/dbenayoun/Generative-AI-Module-Dauphine/blob/main/copie_de_adversarial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from random import random

cuda = torch.cuda.is_available()

## From adversarial examples to training robust models

In the previous notebooks, we focused on methods for solving the maximization problem over perturbations; that is, to finding the solution to the problem
\begin{equation}
\DeclareMathOperator*{\maximize}{maximize}
\maximize_{\|\delta\| \leq \epsilon} \ell(h_\theta(x + \delta), y).
\end{equation}

In this notebook, we will focus on training a robust classifier. More precisly, we aim at solving following minimization problem, namely Adversarial Training:
\begin{equation}
\DeclareMathOperator*{\minimize}{minimize}
\minimize_\theta \frac{1}{|S|} \sum_{x,y \in S} \max_{\|\delta\| \leq \epsilon} \ell(h_\theta(x + \delta), y).
\end{equation}
The order of the min-max operations is important here.  Specially, the max is inside the minimization, meaning that the adversary (trying to maximize the loss) gets to "move" _second_.  We assume, essentially, that the adversary has full knowledge of the classifier parameters $\theta$, and that they get to specialize their attack to whatever parameters we have chosen in the outer minimization. The goal of the robust optimization formulation, therefore, is to ensure that the model cannot be attacked _even if_ the adversary has full knowledge of the model.  Of course, in practice we may want to make assumptions about the power of the adversary but it can be difficult to pin down a precise definition of what we mean by the "power" of the adversary, so extra care should be taken in evaluating models against possible "realistic" adversaries.

## Exercice 1
1. Train a robust classifier using Adversarial Training with a specific norm
2. Evaluate your classifier on natural and adversarial examples crafted with the norm of the training and other norms
3. Make an analysis and conclude

In [ ]:
# load CIFAR10 dataset
def load_cifar(split, batch_size):
  train = True if split == 'train' else False
  dataset = datasets.CIFAR10("./docs", train=split, download=True, transform=transforms.ToTensor())
  return DataLoader(dataset, batch_size=batch_size, shuffle=train)

batch_size = 100
train_loader = load_cifar('train', batch_size)
test_loader = load_cifar('test', batch_size)

100%|██████████| 170498071/170498071 [00:03<00:00, 44278943.26it/s]


Extracting ./docs/cifar-10-python.tar.gz to ./docs
Files already downloaded and verified


In [ ]:
class ConvModel(torch.nn.Module):

  def __init__(self):
    super(ConvModel, self).__init__()
    # First Convolutional Layer: Input channels = 3, Output channels = 32
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
    # First Max Pooling Layer: Reduces each dimension by a factor of 2
    self.pool1 = nn.MaxPool2d(kernel_size=2)

    # Second Convolutional Layer: Input channels = 32, Output channels = 64
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    # Second Max Pooling Layer: Reduces each dimension by a factor of 2
    self.pool2 = nn.MaxPool2d(kernel_size=2)

    self.fc1 = nn.Linear(64 * 8 * 8, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)  # Assuming 10 output classes


  def forward(self, x):
    # First convolutional block: Conv -> ReLU -> MaxPool
    x = self.pool1(nn.functional.relu(self.conv1(x)))
    # Second convolutional block: Conv -> ReLU -> MaxPool
    x = self.pool2(nn.functional.relu(self.conv2(x)))

    # Flatten the output for the fully connected layers
    x = x.view(-1, 64 * 8 * 8)  # Flattening the output from conv layers

    # First fully connected block: FC -> ReLU
    x = nn.functional.relu(self.fc1(x))
    # Second fully connected block: FC -> ReLU
    x = nn.functional.relu(self.fc2(x))
    # Output layer: FC
    x = self.fc3(x)

    return x

In [ ]:
class FastGradientSignMethod:

  def __init__(self, model, eps):
    self.model = model
    self.eps = eps


  def compute(self, x, y):
    """ Construct FGSM adversarial perturbation for examples x"""
    delta = torch.zeros_like(x, requires_grad=True)
    # code here ...
    criterion = nn.CrossEntropyLoss()
    output = self.model(x + delta)
    loss = criterion(output, y)
    loss.backward()
    # Use variable.grad.detach() to retreive the gradient with respect to a loss
    grad = delta.grad.detach()
    delta = self.eps * grad.sign()

    return delta

class ProjectedGradientDescent:

  def __init__(self, model, eps, alpha, num_iter):
    self.model = model
    self.eps = eps
    self.alpha = alpha
    self.num_iter = num_iter

  def compute(self, x, y):
    """ Construct PGD adversarial pertubration on the examples x."""
    delta = torch.zeros_like(x, requires_grad=True)
    for _ in range(self.num_iter):
      if delta.grad is not None:
        delta.grad.zero_()
      output = self.model(x + delta)
      loss = nn.CrossEntropyLoss()(output, y)
      loss.backward()
      delta.data = delta.data + self.alpha * delta.grad.sign()
      delta.data = torch.clamp(delta.data, -self.eps, self.eps)
      delta.data = torch.clamp(x + delta.data, 0, 1) - x
    delta = delta.detach()

    return delta

class ProjectedGradientDescentl2:

  def __init__(self, model, eps, alpha, num_iter):
    # code here ...
    self.model = model
    self.eps = eps
    self.alpha = alpha
    self.num_iter = num_iter

  def compute(self, x, y):
    """ Construct PGD adversarial pertubration on the examples x."""
    # code here ...
    delta = torch.zeros_like(x, requires_grad=True)
    for _ in range(self.num_iter):
      output = self.model(x + delta)
      loss = nn.CrossEntropyLoss()(output, y)
      loss.backward()
      delta.data = delta.data + self.alpha * delta.grad
      #normalize
      delta.data =  delta.data / torch.norm(delta.data, p=2, dim=(1, 2, 3), keepdim=True)
    delta = delta.detach()

    return delta

class ProjectedGradientDescentLinf:

  def __init__(self, model, eps, alpha, num_iter):
    self.model = model
    self.eps = eps  # Maximum perturbation
    self.alpha = alpha  # Step size
    self.num_iter = num_iter  # Number of iterations

  def compute(self, x, y):
    """ Construct PGD adversarial perturbation on the examples x."""
    delta = torch.zeros_like(x, requires_grad=True)  # Initialize delta to zeros

    for _ in range(self.num_iter):
      # Compute the model output with the current perturbation
      output = self.model(x + delta)
      # Compute the loss and backpropagate
      loss = nn.CrossEntropyLoss()(output, y)
      loss.backward()
      # Update delta by taking a step in the direction of the gradient
      delta.data = delta.data + self.alpha * delta.grad.sign()
      # Project delta back into the L_inf ball of radius eps
      delta.data = torch.clamp(delta.data, -self.eps, self.eps)
      # Zero out gradients before the next iteration
      delta.grad.zero_()
    # Detach delta to return the final adversarial perturbation
    delta = delta.detach()

    return delta


In [ ]:
def adversarial_train_model(model, criterion, optimizer, loader, epochs=5, attack=None, perc_attacked=1, verbose=True):
  """Function to train the model"""

  f = 0
  train_losses = []
  for e in range(epochs):
    for batch_idx, (images, labels) in enumerate(train_loader):
        if cuda:
          images, labels = images.cuda(), labels.cuda()
        f = f + 1
        model.train() # - Set the model to train mode
        optimizer.zero_grad()# - Reset the optimizer

        if attack is None:
          output = model(images)
        else:
          if random() < perc_attacked:
            delta = attack.compute(images, labels)
            output = model(images + delta)
          else:
            output = model(images)

        loss = criterion(output, labels)# - Compute the loss
        loss.backward()# - Backward pass
        optimizer.step()# - Update the weights
    if verbose:
      print('Epoch: {} \tTraining Loss: {:.6f}'.format(e+1, loss.item()))
    train_losses.append(loss.item()) #save loss

  return train_losses


In [ ]:
def eval_model(model, loader, attack=None):
  """Function to evaluate your model on a specific loader"""
  accuracy = 0.
  n_inputs = 0.
  for n_batch, (imgs, labels) in enumerate(loader):
    if cuda:
      imgs, labels = imgs.cuda(), labels.cuda()
    if attack is None:
      outputs = model(imgs)
    else:
      delta = attack.compute(imgs, labels)
      adv = imgs + delta
      outputs = model(adv)
    _, predicted = torch.max(outputs.data, 1)
    accuracy += predicted.eq(labels.data).cpu().sum().numpy()
    n_inputs += imgs.shape[0]
  accuracy /= n_inputs

  if attack is None:
    print('accuracy on testset: {:.4f}'.format(accuracy))
  else:
    print('accuracy on testset with attack: {:.4f}'.format(accuracy))


attack = FastGradientSignMethod(model, 0.1)
eval_model(model, test_loader)
eval_model(model, test_loader, attack)



accuracy on testset: 0.6320
accuracy on testset with attack: 0.8960


In [ ]:
print('Model without attack:')
# adverserial training with PGD
model1 = ConvModel()
if cuda:
  model1 = model1.cuda()
# define your loss
criterion = nn.CrossEntropyLoss()
# define the optimizer
opt = optim.Adam(model1.parameters(), lr=0.001)
adversarial_train_model(model1, criterion, opt, train_loader, None)


print('Model with attack:')
model2 = ConvModel()
if cuda:
  model2 = model2.cuda()
# define your loss
criterion = nn.CrossEntropyLoss()
# define the optimizer
opt = optim.Adam(model2.parameters(), lr=0.001)
# define the attack
attack = FastGradientSignMethod(model2, 0.1)
adversarial_train_model(model2, criterion, opt, train_loader, attack)


Model without attack:
Loss: 1.373399257659912
Loss: 1.4012207984924316
Loss: 0.9337642192840576
Loss: 0.838064968585968
Loss: 0.671073317527771
Model with attack:
Loss: 2.031968832015991
Loss: 1.2997732162475586
Loss: 0.6595293283462524
Loss: 0.5677239894866943
Loss: 0.5406886339187622


[2.031968832015991,
 1.2997732162475586,
 0.6595293283462524,
 0.5677239894866943,
 0.5406886339187622]

In [ ]:
eval_model(model1, test_loader)
eval_model(model1, test_loader, attack)

accuracy on testset: 0.7195
accuracy on testset with attack: 0.0847


In [ ]:
eval_model(model2, test_loader, attack)
eval_model(model2, test_loader, None)

accuracy on testset: 0.6388
accuracy on testset with attack: 0.8166


# PLAN  - Adversarial Attacks on CIFAR-10: Robustness and Tradeoffs

## 1. Initial Analysis: Training Robust Models and Accuracy Tradeoff

### A. Training Strategies for Robustness
- **Standard Training**
  - Description of the ConvModel architecture
  - Training on clean CIFAR-10 dataset
- **Adversarial Training**
  - Incorporation of adversarial examples (FGSM, PGD, PGD\_l2) during training
  - Training process and parameters
- **Defense Mechanisms**
  - Implementation of additional defenses (e.g., gradient masking, regularization techniques)
  - Evaluation of their effectiveness

### B. Evaluating Robustness vs. Accuracy Tradeoff
- **Metrics to Consider**
  - Clean Accuracy: Performance on unaltered test set
  - Robust Accuracy: Performance on adversarially perturbed test set
- **Tradeoff Insights**
  - Analysis of how adversarial training affects clean and robust accuracy
  - Comparison of different adversarial training methods
- **Visualization**
  - Graphs plotting clean vs. robust accuracy for various training strategies
- **Statistical Analysis**
  - Statistical significance of observed tradeoffs
  - Discussion on reasons behind the tradeoff

## 2. Elaboration Topic: Transferability of Adversarial Attacks

### A. Understanding Transferability
- **Definition**
  - Explanation of transferability in the context of adversarial attacks
- **Mechanisms**
  - How adversarial examples crafted for one model can affect others

### B. Importance of Studying Transferability
- **Black-Box Attacks**
  - Utilization of transferability for attacks without access to target model parameters
- **Robustness Implications**
  - Impact on the general robustness of different models
- **Security Considerations**
  - Implications for the security of deployed machine learning systems

### C. Experimental Approach
- **Multiple Models**
  - Training diverse ConvModels with varying architectures and hyperparameters
  - Inclusion of models with different adversarial training methods
- **Generating Adversarial Examples**
  - Creation of adversarial examples using FGSM, PGD, and PGD\_l2 on source models
  - Testing these examples on target models to assess transferability
- **Metrics**
  - Transfer Success Rate: Percentage of adversarial examples deceiving target models
  - Robust Accuracy Across Models: Measuring general robustness
- **Factors Influencing Transferability**
  - Model Similarity: Impact of architectural similarities
  - Training Methods: Effect of different defense strategies
  - Attack Strength: Role of perturbation magnitude in transferability

# Adversarial Attacks on CIFAR-10: Robustness and Tradeoffs


## 1. Initial Analysis: Training Robust Models and Accuracy Tradeoff


### A. Training Strategies for Robustness

#### **Standard Training**

In [ ]:
print('Model without attack:')
# adverserial training with PGD
model_noattack = ConvModel()
if cuda:
  model_noattack = model_noattack.cuda()
# define your loss
criterion = nn.CrossEntropyLoss()
# define the optimizer
opt = optim.Adam(model_noattack.parameters(), lr=0.001)
#define attack
attack = None
#define % attacked of the train dataset
perc_attacked = 1
# define epoch
epochs = 20

train_losses = adversarial_train_model(model_noattack, criterion, opt, train_loader, epochs, attack, perc_attacked, verbose=True)

Model without attack:
Epoch: 1 	Training Loss: 1.220773
Epoch: 2 	Training Loss: 1.208490
Epoch: 3 	Training Loss: 0.930145
Epoch: 4 	Training Loss: 0.888815
Epoch: 5 	Training Loss: 0.688768
Epoch: 6 	Training Loss: 0.787551
Epoch: 7 	Training Loss: 0.650179
Epoch: 8 	Training Loss: 0.621199
Epoch: 9 	Training Loss: 0.622477
Epoch: 10 	Training Loss: 0.509969
Epoch: 11 	Training Loss: 0.527332
Epoch: 12 	Training Loss: 0.458468
Epoch: 13 	Training Loss: 0.352289
Epoch: 14 	Training Loss: 0.345791
Epoch: 15 	Training Loss: 0.549821
Epoch: 16 	Training Loss: 0.326741
Epoch: 17 	Training Loss: 0.381404
Epoch: 18 	Training Loss: 0.274787
Epoch: 19 	Training Loss: 0.315009
Epoch: 20 	Training Loss: 0.193689


#### **Adversarial Training**

1. FGSM

In [ ]:
print('Model with attack FGSM:')
# adverserial training with PGD
model_withattackFGSM = ConvModel()
if cuda:
  model_withattack = model_withattackFGSM.cuda()
# define your loss
criterion = nn.CrossEntropyLoss()
# define the optimizer
opt = optim.Adam(model_withattackFGSM.parameters(), lr=0.001)
#define attack
eps = 0.1
fgsm = FastGradientSignMethod(model_withattackFGSM, eps)
#define % attacked of the train dataset
perc_attacked = 1
# define epoch
epochs = 20

train_losses = adversarial_train_model(model_withattackFGSM, criterion, opt, train_loader, epochs, attack=fgsm, perc_attacked=perc_attacked, verbose=True)

Model with attack FGSM:
Epoch: 1 	Training Loss: 2.104448
Epoch: 2 	Training Loss: 1.028874


KeyboardInterrupt: 

2. PGB L0

In [ ]:
print('Model with attack PGD L0:')
# adverserial training with PGD
model_withattackPGD = ConvModel()
if cuda:
  model_withattack = model_withattackPGD.cuda()
# define your loss
criterion = nn.CrossEntropyLoss()
# define the optimizer
opt = optim.Adam(model_withattackPGD.parameters(), lr=0.001)
#define attack
eps = 0.1
pdg_l0 = ProjectedGradientDescent(model_withattackPGD, 0.1, 0.01, 10)
#define % attacked of the train dataset
perc_attacked = 1
# define epoch
epochs = 20

train_losses = adversarial_train_model(model_withattackPGD, criterion, opt, train_loader, epochs, attack=pdg_l0, perc_attacked=perc_attacked, verbose=True)

Model with attack PGD L0:
Epoch: 1 	Training Loss: 2.363302
Epoch: 2 	Training Loss: 2.344585
Epoch: 3 	Training Loss: 2.426562
Epoch: 4 	Training Loss: 2.352633
Epoch: 5 	Training Loss: 2.374098
Epoch: 6 	Training Loss: 2.421401
Epoch: 7 	Training Loss: 2.363866
Epoch: 8 	Training Loss: 2.355221
Epoch: 9 	Training Loss: 2.335323
Epoch: 10 	Training Loss: 2.377704
Epoch: 11 	Training Loss: 2.407084
Epoch: 12 	Training Loss: 2.328311
Epoch: 13 	Training Loss: 2.343179
Epoch: 14 	Training Loss: 2.382799
Epoch: 15 	Training Loss: 2.402846
Epoch: 16 	Training Loss: 2.465587
Epoch: 17 	Training Loss: 2.433835
Epoch: 18 	Training Loss: 2.364945
Epoch: 19 	Training Loss: 2.214850
Epoch: 20 	Training Loss: 2.151494


3. PGD L2

In [ ]:
print('Model with attack PGD L2:')
# adverserial training with PGD
model_withattackPGDL2 = ConvModel()
if cuda:
  model_withattack = model_withattackPGDL2.cuda()
# define your loss
criterion = nn.CrossEntropyLoss()
# define the optimizer
opt = optim.Adam(model_withattackPGDL2.parameters(), lr=0.001)
#define attack
eps = 0.1
pdg_l2 = ProjectedGradientDescent(model_withattackPGDL2, 0.1, 0.01, 10)
#define % attacked of the train dataset
perc_attacked = 1
# define epoch
epochs = 20

train_losses = adversarial_train_model(model_withattackPGDL2, criterion, opt, train_loader, epochs, attack=pdg_l2, perc_attacked=perc_attacked, verbose=True)

Model with attack PGD L2:


KeyboardInterrupt: 

4. PGD Linf


In [ ]:
print('Model with attack PGD Linf:')
# adverserial training with PGD
model_withattackPGDLinf = ConvModel()
if cuda:
  model_withattack = model_withattackPGDLinf.cuda()
# define your loss
criterion = nn.CrossEntropyLoss()
# define the optimizer
opt = optim.Adam(model_withattackPGDLinf.parameters(), lr=0.001)
#define attack
eps = 0.1
pdg_linf = ProjectedGradientDescent(model_withattackPGDLinf, 0.1, 0.01, 5)
#define % attacked of the train dataset
perc_attacked = 1
# define epoch
epochs = 20

train_losses = adversarial_train_model(model_withattackPGDLinf, criterion, opt, train_loader, epochs, attack=pdg_linf, perc_attacked=perc_attacked, verbose=True)

Model with attack PGD Linf:
Epoch: 1 	Training Loss: 1.318361
Epoch: 2 	Training Loss: 1.215485
Epoch: 3 	Training Loss: 0.973842
Epoch: 4 	Training Loss: 0.994524
Epoch: 5 	Training Loss: 1.066445
Epoch: 6 	Training Loss: 1.026074
Epoch: 7 	Training Loss: 1.007287
Epoch: 8 	Training Loss: 0.888043
Epoch: 9 	Training Loss: 0.828196
Epoch: 10 	Training Loss: 0.737218
Epoch: 11 	Training Loss: 0.644678
Epoch: 12 	Training Loss: 0.717615
Epoch: 13 	Training Loss: 0.544334
Epoch: 14 	Training Loss: 0.591962
Epoch: 15 	Training Loss: 0.606081
Epoch: 16 	Training Loss: 0.743542
Epoch: 17 	Training Loss: 0.779675
Epoch: 18 	Training Loss: 0.464911
Epoch: 19 	Training Loss: 0.426206
Epoch: 20 	Training Loss: 0.323298


### B. Evaluating Robustness vs. Accuracy Tradeoff

**Metrics to Consider**

In [ ]:
def eval_model(model, loader, attack=None, attack_name='', model_name=''):
  """Function to evaluate your model on a specific loader"""
  accuracy = 0.
  n_inputs = 0.
  for n_batch, (imgs, labels) in enumerate(loader):
    if cuda:
      imgs, labels = imgs.cuda(), labels.cuda()
    if attack is None:
      outputs = model(imgs)
    else:
      delta = attack.compute(imgs, labels)
      adv = imgs + delta
      outputs = model(adv)
    _, predicted = torch.max(outputs.data, 1)
    accuracy += predicted.eq(labels.data).cpu().sum().numpy()
    n_inputs += imgs.shape[0]
  accuracy /= n_inputs

  if attack is None:
    print('Clean Accuracy, model {} on testset {}: {:.4f}'.format(model_name, attack_name, accuracy))
  else:
    print('Robust Accuracy, model {} on testset with attack {}: {:.4f}'.format(model_name, attack_name, accuracy))


**Tradeoff Insights**


In [ ]:
#Baseline Clean Accuracy
eval_model(model_noattack, test_loader, attack=None, attack_name='', model_name='baseline')


Clean Accuracy, model baseline on testset : 0.9568


In [ ]:
#Robust Accuracy: eval model attacked test set
eval_model(model_noattack, test_loader, attack=fgsm, attack_name='FGSM', model_name='baseline')
eval_model(model_noattack, test_loader, attack=pdg_l0, attack_name='PGD_L0', model_name='baseline')
eval_model(model_noattack, test_loader, attack=pdg_l2, attack_name='PGD_L2', model_name='baseline')
eval_model(model_noattack, test_loader, attack=pdg_linf, attack_name='PGD_Linf',model_name='baseline')

Robust Accuracy, model baseline on testset with attack FGSM: 0.2108
Robust Accuracy, model baseline on testset with attack PGD_L0: 0.3330
Robust Accuracy, model baseline on testset with attack PGD_L2: 0.7860
Robust Accuracy, model baseline on testset with attack PGD_Linf: 0.9422


In [ ]:
#Clean Accuracy: eval model on not attacked test set
eval_model(model_withattackFGSM, test_loader, attack=None, attack_name='', model_name='FGSM')
eval_model(model_withattackPGD, test_loader, attack=None, attack_name='', model_name='PGD_L0')
eval_model(model_withattackPGDL2, test_loader, attack=None, attack_name='', model_name='PGD_L2')
eval_model(model_withattackPGDLinf, test_loader, attack=None, attack_name='', model_name='PGD_Linf')

#Robust Accuracy: eval model attacked test set
eval_model(model_withattackFGSM, test_loader, attack=fgsm, attack_name='FGSM', model_name='FGSM')
eval_model(model_withattackPGD, test_loader, attack=pdg_l0, attack_name='PGD_L0', model_name='PGD_L0')
eval_model(model_withattackPGDL2, test_loader, attack=pdg_l2, attack_name='PGD_L2', model_name='PGD_L2')
eval_model(model_withattackPGDLinf, test_loader, attack=pdg_linf, attack_name='PGD_Linf', model_name='PGD_Linf')

Clean Accuracy, model FGSM on testset : 0.5326
Clean Accuracy, model PGD_L0 on testset : 0.4327
Clean Accuracy, model PGD_L2 on testset : 0.1353
Clean Accuracy, model PGD_Linf on testset : 0.9434
Robust Accuracy, model FGSM on testset with attack FGSM: 0.6686
Robust Accuracy, model PGD_L0 on testset with attack PGD_L0: 0.1165
Robust Accuracy, model PGD_L2 on testset with attack PGD_L2: 0.0890
Robust Accuracy, model PGD_Linf on testset with attack PGD_Linf: 0.8491
